In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import BatchNormalization, Dropout

In [ ]:
def load_and_preprocess_images(folders, image_size=(128, 128)):
    images = []
    labels = []
    label_map = {folder: idx for idx, folder in enumerate(folders)}

    for folder in folders:
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, image_size)
                img = img / 255.0  # Normalize pixel values
                images.append(img)
                labels.append(label_map[folder])

    images = np.array(images)
    labels = to_categorical(labels, num_classes=len(folders))

    return images, labels

In [ ]:
base = "/content/drive/MyDrive/2023_2/Caltech_101/101_ObjectCategories/"
folders = [base + 'tick',base +  'trilobite',base +  'umbrella',base +  'watch',base +  'water_lilly', base + 'wheelchair', base + 'wild_cat',base +  'windsor_chair', base + 'wrench', base + 'yin_yang']


In [ ]:
image_size = (128, 128)
images, labels = load_and_preprocess_images(folders, image_size)

In [ ]:
def build_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Create CNN model
model = build_cnn(input_shape=image_size + (3,), num_classes=len(folders))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 128)               16257152  
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Model training
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)
# Performance evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


NameError: ignored

In [ ]:
import matplotlib.pyplot as plt

# Loss graph
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Accuracy graph
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


NameError: ignored

In [ ]:
"""
Various network architectures: the more complex the model, the better the performance, but if there are too many layers, the performance decreases.
Batch normalization technique: Using batch normalization, model learning becomes stable and performance improves.
Dropout and regularization: improves performance and can also prevent overfitting.
Loss function comparison: Performance varies depending on the loss function, and this can be easily confirmed.
Neural network compression: Compressing the model reduces performance, but reduces capacity and speeds up learning and inference times.
Learning rate and batch size: The larger the batch size, the faster the model can be learned, but the more unstable the learning becomes.
Weight initialization: Using a pre-trained CNN can converge the loss faster.
"""